In [63]:
import sys, os
import json
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.append(os.path.join(os.getcwd(),'../utils'))
from s3 import s3store
from jsonutil import WriteDictJson, ReadDictJson

In [64]:
output = widgets.Output()
clear_output = widgets.Button(description='Clear Output')

test_select = widgets.Select(
    options=[],
    description='Test:',
    disabled=False,
    #rows=25,
)

In [65]:
trainingset='2021-02-22-14-17-19-cocoseg'

In [66]:
creds = ReadDictJson('../creds.json')
if not creds:
    output.append_stdout('Failed to load credentials file {}\n'.format(args.credentails))

s3def = creds['s3'][0]
s3 = s3store(s3def['address'], s3def['access key'], s3def['secret key'], cert_verify=False)
validation = s3def['sets']['trainingset']

test_obj = '{}/{}/tests.json'.format(validation['prefix'], trainingset)
desc_obj = '{}/{}/description.json'.format(validation['prefix'], trainingset)
testjson = s3.GetDict(validation['bucket'], test_obj)
descjson = s3.GetDict(validation['bucket'], desc_obj)

tests = pd.DataFrame(testjson)
display(tests)

,date,name,description,model,accuracy,class_similarity,similarity,confusion,images,image time,batch size,store address,test bucket,platform
0,"03/26/2021, 16:56:04",Test tesorflow inference,Tensorflow image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.947786,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.671104,"[[1019548947, 9595635, 4203804, 2923134], [191...",5000,0.008239,1,ipc.larson.myds.me:31994,mllib,Linux-4.15.0-137-generic-x86_64-with-glibc2.29
1,"03/26/2021, 10:03:51",Test TensorRT inference on Jetson AGX,Tensorflow image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.945007,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.665993,"[[1019350248, 9796639, 4241524, 2959965], [191...",5000,0.029665,1,192.168.0.155:31994,mllib,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...
2,"03/26/2021, 10:09:27",Test Tensorflow inference on Jetson NX,TensorRT image segmentation inference test,NaN,0.941661,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.666916,"[[1019907044, 9629625, 4206098, 2848148], [193...",5000,0.370289,1,192.168.0.155:31994,mllib,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...
3,"03/26/2021, 17:14:46",Test Onnx Inference,Onnx image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.947474,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.667503,"[[1019719461, 9622954, 4165141, 2917806], [192...",5000,0.153856,1,ipc.larson.myds.me:31994,mllib,Linux-4.15.0-137-generic-x86_64-with-glibc2.29
4,"03/26/2021, 17:24:13",Test TensorRT inference,TensorRT image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.947143,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.667975,"[[1019437029, 9934886, 4135475, 2943224], [191...",5000,0.081521,1,ipc.larson.myds.me:31994,mllib,Linux-4.15.0-137-generic-x86_64-with-glibc2.29
5,"03/26/2021, 10:24:28",Test TensorRT inference on Jetson NX,Tensorflow image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.946283,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668803,"[[1019144591, 9674309, 4285973, 2939232], [191...",5000,0.046575,1,192.168.0.155:31994,mllib,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...
6,"03/26/2021, 17:52:09",Test TensorRT inference,TensorRT image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.947243,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.667738,"[[1019966186, 9665795, 4106383, 2881045], [191...",5000,0.078562,1,ipc.larson.myds.me:31994,mllib,Linux-4.15.0-137-generic-x86_64-with-glibc2.29
7,"03/26/2021, 11:07:13",Test Tensorflow inference on Jetson AGX,TensorRT image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.945176,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668743,"[[1019732100, 9768806, 4132898, 2916726], [190...",5000,0.231636,1,192.168.0.155:31994,mllib,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...
8,"03/26/2021, 11:12:48",Test TensorRT inference,TensorRT image segmentation inference test,2021-02-24-10-28-35-cocoseg,0.947821,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668721,"[[1020011948, 9752605, 4045508, 2867675], [191...",5000,0.076331,1,ipc.larson.myds.me:31994,mllib,Linux-4.15.0-137-generic-x86_64-with-Ubuntu-18...


In [67]:
iClasses = {}
classNames = [''] * descjson['classes']['classes']
for clasdesc in descjson['classes']['objects']:
    if clasdesc['trainId'] not in iClasses.keys():
        classNames[clasdesc['trainId']] = clasdesc['category']
        if not (clasdesc['category'] == 'void'):
            iClasses[clasdesc['trainId']] = clasdesc

overview = {}
results = {}
confusion = {}
test_names = []
    
for test in testjson:
    if 'name' in test:
        name = '{} {}'.format(test['name'], test['date'])
    elif 'model' in test:
        name = '{} {}'.format(test['model'], test['date'])
    else: 
        name = '{}'.format(test['date'])
    test_names.append(name)
    
    test_overview = {'images':test['images'],'accuracy':test['accuracy'],'similarity':test['similarity'], 'inference time':test['image time']}
    overview[name]=test_overview
    
    similarity = {}
    for key in iClasses:
        res = test['class_similarity'][str(key)]
        if res:
            similarity[iClasses[key]['category']] = test['class_similarity'][str(key)]['similarity']

    results[name] = {'similarity':similarity, 'confusion':test['confusion']}
    
display(pd.DataFrame(overview).T)

,images,accuracy,similarity,inference time
"Test tesorflow inference 03/26/2021, 16:56:04",5000.0,0.947786,0.671104,0.008239
"Test TensorRT inference on Jetson AGX 03/26/2021, 10:03:51",5000.0,0.945007,0.665993,0.029665
"Test Tensorflow inference on Jetson NX 03/26/2021, 10:09:27",5000.0,0.941661,0.666916,0.370289
"Test Onnx Inference 03/26/2021, 17:14:46",5000.0,0.947474,0.667503,0.153856
"Test TensorRT inference 03/26/2021, 17:24:13",5000.0,0.947143,0.667975,0.081521
"Test TensorRT inference on Jetson NX 03/26/2021, 10:24:28",5000.0,0.946283,0.668803,0.046575
"Test TensorRT inference 03/26/2021, 17:52:09",5000.0,0.947243,0.667738,0.078562
"Test Tensorflow inference on Jetson AGX 03/26/2021, 11:07:13",5000.0,0.945176,0.668743,0.231636
"Test TensorRT inference 03/26/2021, 11:12:48",5000.0,0.947821,0.668721,0.076331


In [68]:
c = np.array(results[test_names[0]]['confusion'])
norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
norm_confusion = norm_confusion.round(decimals=3)

confusion_text = [[str(round(y, 3)) for y in x] for x in norm_confusion]

# set up figure 
confusion_plot = plotly.figure_factory.create_annotated_heatmap(norm_confusion, x=classNames, y=classNames, colorscale='plasma')

# add custom xaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=0.5,
                   y=-0.15,
                   showarrow=False,
                   text="Predicted value",
                   xref="paper",
                   yref="paper"))

# add custom yaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=-0.2,
                   y=0.5,
                   showarrow=False,
                   text="Real value",
                   textangle=-90,
                   xref="paper",
                   yref="paper"))


# adjust margins to make room for yaxis title
confusion_plot.update_layout(margin=dict(t=50, l=200))

# add colorbar
confusion_plot['data'][0]['showscale'] = True
confusion_plot.update_layout(yaxis_autorange="reversed")

confusion_display = graph_obj.FigureWidget(confusion_plot)


In [69]:
def SelectTest(change, select, display, testjson):
    global test_select
    global confusion_display
    global output
    global results
    global confusion_plot
    
    with confusion_display.batch_update():
        
        c = np.array(results[test_select.value]['confusion'])
        norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
        confusion_text = [str(round(x, 3)) for x in norm_confusion.flatten()]
        confusion_display.data[0].z =  norm_confusion
        confusion_display.update_layout(yaxis_autorange="reversed")
        
        for i in range(len(confusion_text)):
            confusion_display.layout.annotations[i].text = confusion_text[i]
            
def ClearOutput(b):
    global test_select
    global confusion_display
    global output
    output.clear_output()

In [70]:

clear_output.on_click(lambda b: ClearOutput(b))


display_results = widgets.HBox([test_select, confusion_display])
attach = widgets.VBox([display_results, clear_output, output])

test_select.observe (lambda change:SelectTest(change, select=test_select, display=confusion_display, testjson=testjson), names="value")

if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]

display(attach)